# Torchvision using torch.compile with OpenVINO backend

This is a self-contained example that relies solely on its own code.

<img referrerpolicy="no-referrer-when-downgrade" src="https://static.scarf.sh/a.png?x-pxid=5b5a4db0-7875-4bfb-bdbd-01698b5b1a77&file=notebooks/latent-consistency-models-image-generation/latent-consistency-models-optimum-demo.ipynb" />


### Prerequisites
[back to top ⬆️](#Table-of-contents:)

Install required packages

In [1]:
import sys
!{sys.executable} -m pip install -q "openvino>=2023.3.0"
!{sys.executable} -m pip install -q "torchvision" "pillow" "urllib3" "torch>=2.1.1"

In [2]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import urllib

# Load the Inception v3 model pre-trained on ImageNet
model = models.inception_v3(pretrained=True)
model.eval()  # Set the model to evaluation mode

model = torch.compile(model,backend="openvino", options = {"device" : "CPU"})

# Define the image preprocessing steps
preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Path to the image you want to classify

url, img_path = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
urllib.request.urlretrieve(url, img_path)
# Load the image
img = Image.open(img_path)

# Preprocess the image
img_tensor = preprocess(img)

# Add a batch dimension (1, 3, 299, 299)
img_tensor = img_tensor.unsqueeze(0)


In [3]:
# Make predictions
with torch.no_grad():
    outputs = model(img_tensor)

# Apply softmax to get probabilities
probabilities = torch.nn.functional.softmax(outputs[0], dim=0)

# Get the top 3 predictions
top_probabilities, top_indices = torch.topk(probabilities, 3)

# Load the labels for ImageNet classes
LABELS_URL = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
import urllib
import json
with urllib.request.urlopen(LABELS_URL) as url:
    labels = json.loads(url.read().decode())

# Print the top 3 predictions with probabilities
print('Predictions:')
for i in range(top_indices.size(0)):
    label = labels[top_indices[i]]
    prob = top_probabilities[i].item()
    print(f"{i+1}: {label} ({prob:.4f})")

img